# BRFSS Depression Indicators Dashboard (Python + Dash)

This project implements an interactive dashboard for exploring the Behavioral Risk Factor Surveillance System (BRFSS) chronic health indicators dataset. The dashboard allows users to select any BRFSS survey question and view confidence interval summaries across multiple demographic and geographic breakouts.

The system is designed to handle a very large dataset (~700MB CSV) efficiently by converting it into Apache Parquet format, which dramatically improves loading time and memory efficiency. All processing and aggregation are done in Python using `pandas`, and the visual interface is built using `Dash` and `Plotly`.

The goal of this project is to recreate, in Python, the analytical workflow originally demonstrated in R/Shiny, while improving performance, modularity, and clarity. The final result is an interactive dashboard with tab-based navigation and consistent confidence-interval computations across all demographic dimensions.


# Workflow Design and Rationale

The overall design of the dashboard follows the structured workflow outlined in the project instructions:

### 1. **Data Preparation (Parquet Format)**
The original BRFSS CSV file is large and slow to query. To optimize performance, the dataset is converted into Parquet format, which supports faster reads, better compression, and more efficient filtering. All downstream analysis uses `brfss.parquet`.

### 2. **Question Selection Tool**
BRFSS contains many survey questions grouped under high-level categories such as Class, Topic, and Question.  
To make navigation intuitive, the dashboard includes a three-stage selection tool:
- **Class** → filters available Topics  
- **Topic** → filters available Questions  
- **Question** → triggers all subpanel updates

This mirrors the structure of the BRFSS metadata and prevents information overload.

### 3. **Aggregation Workflow**
For each selected question, the system:
- Filters the relevant observations
- Standardizes inconsistent labels (ResponseID, BreakoutID)
- Merges categories where BRFSS changed definitions across years
- Computes sample-size–weighted percentages
- Computes 95% confidence intervals using  
  \[
  CI = p \pm 2\sqrt{p(1-p)/n}
  \]

A universal aggregation function (`compute_ci`) is applied across all breakouts to ensure uniform calculations.

### 4. **Dashboard Subpanels**
The dashboard follows the required structure:
- **Overall**
- **Temporal (Year)**
- **By Gender**
- **By Age Group**
- **By Education**
- **By Income**
- **By Race**
- **By Location (State/Territory)**

Each subpanel focuses on exactly one breakout dimension, showing the distribution of responses with confidence intervals.

### 5. **More/Less Options**
Certain demographic categories (e.g., Income, Age, Race) can be shown in full detail ("More") or in simplified merged groups ("Less").  
This is implemented with an optional category-mapping layer, activated through radio-button toggles.

The combination of Parquet data, modular utilities, consistent CI computation, and Dash interactivity creates a scalable, professional-quality analytical dashboard aligned with the project specifications.


In [1]:
# For he coding part we have decided t go with this workflow

# brfss_dash_app/
# │
# ├── app.py                 ← Main Dash app
# ├── data/
# │    └── brfss.parquet     ← Optimized dataset
# │
# ├── utils/
# │    ├── merges.py         ← ResponseID & BreakoutID merge functions
# │    ├── prepare.py        ← prepare_qDf() & CI logic
# │    ├── agg.py            ← aggregation functions for each subpanel
# │    └── options.py        ← “more / less” category mappings
# │
# └── assets/
#      └── styles.css        ← (optional) styling


In [2]:
# Dash layout will include:

#     Class -> Topic -> Question

#     The user will have the option to select Class
#     The Topic part will have dropdown Options
#     The Question dropdown updates
#     Dashboard updates will occur across all panels automatically

# For the Automation we will use a decorator something like this:
# @app.callback(...)
# def update_topcis(selected_class):
#     ...

In [3]:
# Designing the aggragation workflow:

# Implementation:

#     We will build one universal Pandas function to:

#     Filter rows for the selected question 

#     Merge category codes (ResponseID, BreakOutID)

#     Group by chosen breakout type

#     Compute sample-size weighted percentages

#     Compute 95% CI (Confidence Interval):

# The formula for Confidence looks like: CI = p +- 2 * sqrt(p(1-p)/n)


# And so this becomes:

#     utils/agg.py:

#     def compute_ci(df, group_var):
#     grouped = ...
#     grouped["percent"] = ...
#     grouped["ci_low"] = ...
#     grouped["ci_high"] = ...
#     return grouped



# Each panel calls this function with different group_var



In [4]:
# Designing the Overall subpanel without 'more' or 'less' options

# This is the base version where: 

#     ✔ All responses are shown individually
#     ✔ Bar chart or stacked bar are formed
#     ✔ Confidence intervals are computed
#     ✔ We don't have any category merging

# For this we will implement:

#     dcc.Graph('id = overall')

# The callback will look like:
    
#     @app.callback(Output("overall", "figure"), Input("question", "value"))
#     def update_overall(q):
#         qDf = prepare_qDf(df, q)
#         agg = compute_ci(qDf[q_category("overall")])
#         fig = make_bar(agg)


In [5]:
# If we were to augment the Overall subpanel with 'more' or 'less' options.

# We add: (something like)
#     Radio buttons
#     More -> detailed
#     Less -> merged categories

# Example migh look something like this:

#     dcc.RadioItems(
#        id="overall_mode",
#        options=["more", "less"],
#        value="more",
#        inline=True
#     )

# Call back Changes Behavior:
#     if mode == "less":
#         agg = merge_categories(agg)

# This should satisfy 'More vs Less' requirement.


In [6]:
# Desigining other subpanels without 'more' or 'less' options.

# We shall first implement:

# -By_Age
# -By_Gender
# -By_Race
# -By_Education
# -By_Income
# -By_Location
# -Temporal

# Each one will have its own graph.

In [7]:
# After modifying these subpanels to include ‘more’ or ‘less’ options.”

# We basically repeat the same pattern:

# Add small radio buttons

# Collapse categories if needed

# Examples:

# - Age merged

# “18-24, 25-34, 35-44, 45+”

# - Education merged

# HS or less / Some college / College+

# - Gender merged

# Male/Female only (hide “Unknown”)

# - Race merged

# White / Black / Hispanic / Other

# This should do the trick

In [8]:
# The final dashboard aims to finish with:

# ✔ Loading the Parquet data
# ✔ Letting the users choose any question that they want
# ✔ Being able to show 7 analytic panels
# ✔ To be able to Compute confidence intervals
# ✔ Support for the “More” vs “Less” modes

In [1]:
import pandas as pd  # imports the pandas module

In [2]:
csv_path = '/Users/swapnanilbala/Documents/Git_Hub_Repos/brfss_dash_app/Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_20251129.csv'

chunk = pd.read_csv(csv_path, chunksize=100000)

first_chunk = next(chunk)
first_chunk.head()

print(first_chunk.columns)


Index(['Year', 'Locationabbr', 'Locationdesc', 'Class', 'Topic', 'Question',
       'Response', 'Break_Out', 'Break_Out_Category', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'DataSource',
       'ClassId', 'TopicId', 'LocationID', 'BreakoutID', 'BreakOutCategoryID',
       'QuestionID', 'ResponseID', 'GeoLocation'],
      dtype='object')


In [21]:
first_chunk.tail()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
99995,2023,MI,Michigan,Health Status,Overall Health,How is your general health?,Poor,"Less than $15,000",Household Income,76,...,NaN,BRFSS,CLASS08,TOPIC41,26,INCOME01,CAT6,GENHLTH,RESP060,"(44.6613195430005, -84.71439026999968)"
99996,2023,MI,Michigan,Health Status,Overall Health,How is your general health?,Excellent,"$15,000-$24,999",Household Income,46,...,NaN,BRFSS,CLASS08,TOPIC41,26,INCOME02,CAT6,GENHLTH,RESP056,"(44.6613195430005, -84.71439026999968)"
99997,2023,MI,Michigan,Health Status,Overall Health,How is your general health?,Very good,"$15,000-$24,999",Household Income,150,...,NaN,BRFSS,CLASS08,TOPIC41,26,INCOME02,CAT6,GENHLTH,RESP057,"(44.6613195430005, -84.71439026999968)"
99998,2023,MI,Michigan,Health Status,Overall Health,How is your general health?,Good,"$15,000-$24,999",Household Income,299,...,NaN,BRFSS,CLASS08,TOPIC41,26,INCOME02,CAT6,GENHLTH,RESP058,"(44.6613195430005, -84.71439026999968)"
99999,2023,MI,Michigan,Health Status,Overall Health,How is your general health?,Fair,"$15,000-$24,999",Household Income,170,...,NaN,BRFSS,CLASS08,TOPIC41,26,INCOME02,CAT6,GENHLTH,RESP059,"(44.6613195430005, -84.71439026999968)"


In [23]:
poor = first_chunk[['Response']].value_counts()
poor

Response                                               
No                                                         21216
Yes                                                        20600
Have some form of health insurance                          1210
Do not have some form of health insurance                   1210
Never                                                        644
                                                           ...  
Black, non-Hispanic                                          440
Native Hawaiian or other Pacific Islander, non-Hispanic      440
Other race, non-Hispanic                                     440
Hispanic                                                     440
MultiRacial, non-Hispanic                                    440
Name: count, Length: 98, dtype: int64

In [20]:
# Here we have decided to take a look at the datatypes that our dataframe holds
first_chunk.describe()

,Year,Data_value,Confidence_limit_Low,Confidence_limit_High,LocationID
count,100000.0,79669.000000,79669.000000,79669.000000,100000.000000
mean,2023.0,40.643374,37.137237,44.139815,15.909100
std,0.0,33.642491,33.439960,34.086882,13.000859
min,2023.0,0.200000,0.100000,0.300000,1.000000
25%,2023.0,10.200000,7.400000,12.800000,8.000000
50%,2023.0,29.300000,24.100000,34.200000,15.000000
75%,2023.0,73.800000,68.600000,78.900000,20.000000
max,2023.0,100.000000,100.000000,100.000000,66.000000


In [4]:
first_chunk['Question'].unique()

array(['Ever told you that you have a form of depression?',
       'What is your age?',
       'Are you blind or do you have serious difficulty seeing, even when wearing glasses?',
       'Ever told you have COPD?', 'Ever told you have kidney disease?',
       'Ever told you had any other types of cancer?',
       'Ever told you had skin cancer?',
       'About how long has it been since you last visited a doctor for a routine checkup?',
       'Ever told you had angina or coronary heart disease?',
       'Ever told you had a heart attack (myocardial infarction)?',
       'Ever told you had a stroke?', 'Adults who reported being deaf',
       'Do you have serious difficulty concentrating, remembering, or making decisions?',
       'Have you ever been told by a doctor that you have diabetes?',
       'Do you have difficulty doing errands alone?',
       'Do you have difficulty dressing or bathing?',
       'Do you have serious difficulty walking or climbing stairs?',
       'Adults who 

In [24]:
# Parquet Conversion

csv_path = '/Users/swapnanilbala/Documents/Git_Hub_Repos/brfss_dash_app/Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_20251129.csv'

print("Loading CSV (this may take a minute)...")

# Load the CSV safely
df = pd.read_csv(csv_path, low_memory=False)

print("Loaded CSV. Selecting essential columns...")

# Keep only columns needed for extraction & aggregation
cols_needed = [
    "Year", 
    "Locationabbr", 
    "Class", 
    "Topic", 
    "Question",
    "Response", 
    "Break_Out", 
    "Break_Out_Category", 
    "Sample_Size",
    "BreakoutID", 
    "BreakOutCategoryID", 
    "ResponseID"
]

df_clean = df[cols_needed].copy()

print("Converting column dtypes...")

# Ensure all string-like columns are consistent
df_clean = df_clean.astype({
    "Locationabbr": "string",
    "Class": "string",
    "Topic": "string",
    "Question": "string",
    "Response": "string",
    "Break_Out": "string",
    "Break_Out_Category": "string",
    "BreakoutID": "string",
    "BreakOutCategoryID": "string",
    "ResponseID": "string",
})

print("Saving to Parquet...")

df_clean.to_parquet("data/brfss.parquet", index=False)

print("\n🔥 SUCCESS! brfss.parquet is ready.\n")



Loading CSV (this may take a minute)...
Loaded CSV. Selecting essential columns...
Converting column dtypes...
Saving to Parquet...

🔥 SUCCESS! brfss.parquet is ready.

